# Kernel Density Estimate

Alessandro Samuel-Rosa, Taciara Zborowski Horst

2025-07-06

License: MIT

In [186]:
import ee
import geemap

try:
    # Initialize the Earth Engine library.
    # Authenticate and initialize GEE if you are running this locally.
    # In some environments like Google Colab, ee.Authenticate() might not be needed.
    # ee.Authenticate()
    ee.Initialize(project='mapbiomas-solos-workspace') # Your project ID
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    # Fallback for environments where initialization might fail but a default project exists
    try:
        ee.Initialize()
    except Exception as e_fallback:
        print(f"Fallback Earth Engine initialization also failed: {e_fallback}")
        # Exit or handle the error as appropriate for your application
        exit()

In [187]:
# 1. Define the input data path
fc_dir = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/'
fc_name = 'matriz-v010_0_10cm'
soil_samples_path = f"{fc_dir}{fc_name}"

In [188]:
# 2. Load the datasets
try:
    # Load the soil sampling points as a FeatureCollection
    soil_samples = ee.FeatureCollection(soil_samples_path)
    # Load Brazil boundary for clipping the final result
    brazil_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
except ee.EEException as e:
    print(f"Error loading data from Earth Engine: {e}")
    exit()

In [189]:
# 3. Define the Heatmap Function
# This function encapsulates the logic from the provided JavaScript example.
def create_heatmap(points_fc, pixel_radius, scale):
    """Generates a heatmap image from a feature collection of points."""

    # Add a dummy property to each feature to be used for rasterization.
    points_with_dummy = points_fc.map(lambda feature: feature.set('dummy', 1))

    # Rasterize the points into an image. Use Reducer.first() and unmask to make
    # a binary image of points (1) and non-points (0).
    # We reproject here to define the scale at which the pixel_radius will be applied.
    point_image = points_with_dummy.reduceToImage(
        properties=['dummy'],
        reducer=ee.Reducer.first()
    ).reproject(
        crs='EPSG:4326',
        scale=scale
    ).unmask(0)

    # Define a circular kernel. The radius is in pixels.
    kernel = ee.Kernel.circle(radius=pixel_radius)

    # Apply the kernel to the point image. Convolve sums the kernel values
    # in the neighborhood of each pixel.
    heatmap = point_image.convolve(kernel)

    # Return the heatmap without masking zero values to create a continuous surface.
    return heatmap

In [190]:
# 4. Generate the Heatmap Image
# Define the scale of the analysis and the radius of the kernel in pixels.
# A larger radius creates a more smoothed/generalized heatmap.
calculation_scale = 1000 # 1km pixels
heatmap_radius_pixels = 20 # a 20-pixel radius at a 1km scale

# Create the heatmap image using the function.
heatmap_image_unclipped = create_heatmap(soil_samples, heatmap_radius_pixels, calculation_scale)

# Clip the final heatmap to the boundary of Brazil.
heatmap_image = heatmap_image_unclipped.clip(brazil_boundary)

In [191]:
# 5. Define visualization parameters for the heatmap
# Using the palette and max value from the example.
heatmap_vis_params = {
    'min': 0,
    'max': 1, # This value may need adjustment for your data density
    'palette': ['lightgreen', 'yellow', 'red']
}

In [192]:
# 6. Create and display the map
# Initialize a geemap Map object
m = geemap.Map(center=[-14, -54], zoom=4) # Center on Brazil

# Add the final heatmap layer to the map
m.addLayer(heatmap_image, heatmap_vis_params, 'Soil Sample Heatmap')

# Add the original soil sample points for reference
m.addLayer(soil_samples, {'color': 'blue'}, 'Soil Sample Points', False) # Initially off

# Add a layer control to toggle layers on and off
m.addLayerControl()

# Display the map
m


Map(center=[-14, -54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…